<a href="https://colab.research.google.com/github/11061995/COE-523-Assignment1/blob/main/efficientvit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Mounting data & repo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -rf Cream  # Remove existing directory if needed
!git clone https://github.com/microsoft/Cream.git
!ls
#cd Cream/EfficientViT/classification

Cloning into 'Cream'...
remote: Enumerating objects: 2175, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 2175 (delta 201), reused 189 (delta 189), pack-reused 1910 (from 2)
Receiving objects: 100% (2175/2175), 8.81 MiB | 25.34 MiB/s, done.
Resolving deltas: 100% (831/831), done.
Cream  data  drive  sample_data


In [ ]:
import torch
from torchvision import datasets, transforms

# Define transforms (CIFAR-10 images are 32x32 pixels)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# Download and load CIFAR-10 datasets
train_dataset = datasets.CIFAR10(
    root='./data',   # Dataset will be saved to /content/data
    train=True,      # Load training set
    download=True,   # Auto-download if not found
    transform=train_transform
)

val_dataset = datasets.CIFAR10(
    root='./data',   # Same root path
    train=False,     # Load validation/test set
    download=True,
    transform=val_transform
)

print("Training samples:", len(train_dataset))    # 50,000 images
print("Validation samples:", len(val_dataset))   # 10,000 images

Training samples: 50000
Validation samples: 10000


### Step 2:Reorganize the validation set to match the train folder structure:

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# Load datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_transform)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

### Step 3: Modify the Configuration File

Edit the config file (configs/efficientvit.yaml) to match your dataset:

In [ ]:
# ✅ Correct (Python dictionary)
config = {
    "model": {
        "type": "efficientvit",
        "input_size": 32  # For CIFAR-10 (32x32 images)
    },
    "data": {
        "name": "cifar10",
        "num_classes": 10
    }
}

In [ ]:
%cd Cream/EfficientViT/classification/model/
!ls  # Verify efficientvit.py exists

/content/Cream/EfficientViT/classification/model
build.py  efficientvit.py  __init__.py


In [ ]:
cd classification/

[Errno 2] No such file or directory: 'classification/'
/content/Cream/EfficientViT/classification/model


### Step 4: install requirement

Modify the dataset loading in train.py (or create a new file datasets/cifar10.py)

In [ ]:
!pip3 install torch torchvision timm einops fvcore easydict matplotlib numpy yacs scikit-image pillow

### Step 5: Adjust Model Architecture

In models/efficientvit.py, ensure the final classification head matches your dataset classes

In [ ]:
!pip install efficientvit
from efficientvit import EfficientViT

# Initialize model with PROPER parameters
model = EfficientViT(
    img_size=32,       # ✅ Correct parameter name (not image_size)
    num_classes=10,      # CIFAR-10 has 10 classes
    patch_size=4,        # 32x32 → 32/4=8 patches per dimension
    embed_dim=[64, 128, 192],  # Reduced dimensions for CIFAR-10
    window_size=[6, 6, 6],     # Smaller window size for 32x32 input
    kernels=[5, 5, 5, 5]
)



ERROR: Could not find a version that satisfies the requirement efficientvit (from versions: none)
ERROR: No matching distribution found for efficientvit


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
model = EfficientViT(img_size=32, num_classes=10)
x = torch.randn(2, 3, 32, 32)  # Test batch
print(model(x).shape)  # Should output [2, 10]

torch.Size([2, 10])


### Step 6: Start Training

In [ ]:
# Change to the correct directory
%cd /content/Cream/EfficientViT/classification/

/content/Cream/EfficientViT/classification


In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

####Create a training loop

In [ ]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(loader)
    train_acc = 100. * correct / total
    return train_loss, train_acc

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = running_loss / len(loader)
    val_acc = 100. * correct / total
    return val_loss, val_acc

####Run the training

In [ ]:
num_epochs = 50
best_acc = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)

    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    # Update scheduler
    scheduler.step()

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved best model!")

    print("-" * 50)

Epoch 1/50


100%|██████████| 79/79 [00:04<00:00, 17.85it/s]


Train Loss: 1.6996 | Train Acc: 37.28%
Val Loss: 1.4624 | Val Acc: 46.33%
Saved best model!
--------------------------------------------------
Epoch 2/50


100%|██████████| 79/79 [00:05<00:00, 14.67it/s]


Train Loss: 1.4531 | Train Acc: 46.92%
Val Loss: 1.3487 | Val Acc: 50.93%
Saved best model!
--------------------------------------------------
Epoch 3/50


...

Train Loss: 0.7076 | Train Acc: 74.93%
Val Loss: 0.7804 | Val Acc: 72.89%
--------------------------------------------------
Epoch 49/50


100%|██████████| 79/79 [00:04<00:00, 17.19it/s]


Train Loss: 0.7032 | Train Acc: 74.73%
Val Loss: 0.7818 | Val Acc: 72.59%
--------------------------------------------------
Epoch 50/50


100%|██████████| 79/79 [00:03<00:00, 21.03it/s]

Train Loss: 0.6969 | Train Acc: 75.13%
Val Loss: 0.7743 | Val Acc: 73.12%
--------------------------------------------------


### Step 7: Evaluate the Model

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation
val_loss, val_acc = validate(model, val_loader, criterion, device)
print(f"Final Validation Accuracy: {val_acc:.2f}%")

100%|██████████| 313/313 [00:09<00:00, 32.34it/s]

Final Validation Accuracy: 73.27%


###Step 8: Visualization

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

class EfficientViTCAM:
    def __init__(self, model):
        self.model = model
        self.features = None
        self.gradients = None
        self.model.eval()

        # Find the last convolutional layer
        def find_target_layer(module):
            target_layer = None
            for name, layer in reversed(list(module.named_modules())):
                if isinstance(layer, torch.nn.Conv2d):
                    target_layer = layer
                    print(f"✅ Found target layer: {name}")
                    break
            return target_layer

        self.target_layer = find_target_layer(self.model)

        if self.target_layer is None:
            raise ValueError("❌ No convolutional layer found for Grad-CAM")

        # Register hooks
        self.target_layer.register_forward_hook(self.save_features)
        self.target_layer.register_full_backward_hook(self.save_gradients)
        print("✅ Hooks registered successfully")

    def save_features(self, module, input, output):
        self.features = output.detach()
        print(f"✅ Features saved - shape: {self.features.shape}")

    def save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
        print(f"✅ Gradients saved - shape: {self.gradients.shape}")

    def generate_cam(self, input_image, target_class=None):
        print("\n=== Generating CAM ===")
        print(f"Input image shape: {input_image.shape}")

        # Forward pass
        output = self.model(input_image.unsqueeze(0))
        print(f"Model output shape: {output.shape}")

        if target_class is None:
            target_class = output.argmax(dim=1).item()
        print(f"Target class: {target_class}")

        # Backward pass
        self.model.zero_grad()
        one_hot = torch.zeros_like(output)
        one_hot[0][target_class] = 1
        output.backward(gradient=one_hot)
        print("✅ Backward pass completed")

        # Process gradients and activations
        pooled_gradients = torch.mean(self.gradients, dim=[0, 2, 3], keepdim=True)
        activations = self.features
        print(f"Pooled gradients shape: {pooled_gradients.shape}")
        print(f"Activations shape: {activations.shape}")

        # Weight activations by gradients
        weighted_activations = activations * pooled_gradients
        print(f"Weighted activations shape: {weighted_activations.shape}")

        # Generate heatmap
        heatmap = torch.mean(weighted_activations, dim=1).squeeze()
        print(f"Heatmap before processing shape: {heatmap.shape}")

        # Convert to numpy and process
        heatmap = heatmap.cpu().numpy()
        print(f"Heatmap numpy shape: {heatmap.shape}")

        heatmap = np.maximum(heatmap, 0)
        print(f"Heatmap after max(0): {heatmap.shape}")

        # Resize to input image dimensions
        heatmap = cv2.resize(heatmap, (input_image.shape[2], input_image.shape[1]))
        print(f"Heatmap after resize: {heatmap.shape}")

        # Normalize
        heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap) + 1e-8)
        print("✅ Heatmap generated successfully")

        return heatmap

def visualize_heatmap(image, heatmap, true_class, pred_class, alpha=0.5):
    print("\n=== Visualizing ===")
    # Convert tensor image to numpy
    img = image.cpu().numpy().transpose(1, 2, 0)
    mean = np.array([0.4914, 0.4822, 0.4465])
    std = np.array([0.2470, 0.2435, 0.2616])
    img = std * img + mean  # Un-normalize
    img = np.clip(img, 0, 1)
    print(f"Image numpy shape: {img.shape}")

    # Create heatmap
    print(f"Heatmap input shape: {heatmap.shape}")
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    print(f"Heatmap after color mapping: {heatmap.shape}")

    # Superimpose heatmap on image
    superimposed_img = cv2.addWeighted(np.uint8(255 * img), alpha, heatmap, 1-alpha, 0)
    print(f"Final output shape: {superimposed_img.shape}")

    # Display
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"True: {true_class}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img)
    plt.title(f"Predicted: {pred_class}")
    plt.axis('off')

    plt.tight_layout()
    print("✅ Showing visualization...")
    plt.show()

def generate_and_visualize_heatmaps(model, data_loader, num_examples=5):
    print("===== Starting Grad-CAM Visualization =====")
    try:
        cam_generator = EfficientViTCAM(model)
    except ValueError as e:
        print(f"❌ Error initializing Grad-CAM: {e}")
        return

    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
              'dog', 'frog', 'horse', 'ship', 'truck']

    # Get some samples
    data_iter = iter(data_loader)
    sample_images, sample_labels = next(data_iter)
    sample_images, sample_labels = sample_images.to(device), sample_labels.to(device)
    print(f"Loaded {len(sample_images)} examples")

    for i in range(min(num_examples, len(sample_images))):
        print(f"\n==== Processing example {i+1}/{min(num_examples, len(sample_images))} ====")
        image = sample_images[i]
        label = sample_labels[i].item()

        # Get prediction
        with torch.no_grad():
            output = model(image.unsqueeze(0))
            pred_class = output.argmax(dim=1).item()
        print(f"True class: {classes[label]}, Predicted: {classes[pred_class]}")

        # Generate heatmap
        try:
            heatmap = cam_generator.generate_cam(image, pred_class)
            # Visualize
            visualize_heatmap(image, heatmap, classes[label], classes[pred_class])
        except Exception as e:
            print(f"❌ Error generating heatmap for example {i}: {e}")

# Run the visualization
print("===== Starting Visualization Process =====")
generate_and_visualize_heatmaps(model, val_loader, num_examples=5)
print("===== Visualization Complete =====")

===== Starting Visualization Process =====
===== Starting Grad-CAM Visualization =====
✅ Found target layer: blocks3.5.ffn1.m.pw2.c
✅ Hooks registered successfully
Loaded 32 examples

==== Processing example 1/5 ====
✅ Features saved - shape: torch.Size([1, 192, 1, 1])
✅ Features saved - shape: torch.Size([1, 192, 1, 1])
True class: cat, Predicted: cat

=== Generating CAM ===
Input image shape: torch.Size([3, 32, 32])
✅ Features saved - shape: torch.Size([1, 192, 1, 1])
✅ Features saved - shape: torch.Size([1, 192, 1, 1])
Model output shape: torch.Size([1, 10])
Target class: 3
✅ Gradients saved - shape: torch.Size([1, 192, 1, 1])
✅ Gradients saved - shape: torch.Size([1, 192, 1, 1])
✅ Backward pass completed
Pooled gradients shape: torch.Size([1, 192, 1, 1])
Activations shape: torch.Size([1, 192, 1, 1])
Weighted activations shape: torch.Size([1, 192, 1, 1])
Heatmap before processing shape: torch.Size([])
Heatmap numpy shape: ()
Heatmap after max(0): ()
❌ Error generating heatmap for ex